In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import nltk
from string import punctuation
from sklearn.decomposition import NMF
import json
from sklearn.metrics.pairwise import cosine_similarity


# Vetorização
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from xgboost import XGBClassifier


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [2]:
stopwords = nltk.corpus.stopwords.words('portuguese')
numbers = '0123456789'

In [3]:
df = pd.read_csv('noticias.csv')
df.tail()

,noticia,target
16309,Desenvolvedores do Ethereum preparam grandes n...,0
16310,Ripple firma-se como a segunda maior criptomoe...,0
16311,Bitcoin se recuperam e ganha impulso de alta\r...,0
16312,"Essas 24 ICOs arrecadaram 2,8 bilhões de dólar...",0
16313,Razões pelas quais o Bitcoin pode valer 6 mil ...,0


In [4]:
df_target = df['target']
df_target.tail()

16309    0
16310    0
16311    0
16312    0
16313    0
Name: target, dtype: int64

In [85]:
def processa(row):
    txt = row['noticia']
    
    return ' '.join([t for t in word_tokenize(txt.lower()) if (t not in stopwords) and (t not in punctuation) and (not t.isdigit())])

df['doc'] = df.apply(processa, axis=1)

df['doc'].head()

0    carona deyverson atacante palmeiras abre casa ...
1    river x boca final libertadores madri bênção f...
2    e-brasileirão assista vivo fase final campeona...
3    inauguração estátua renato gaúcho grêmio fica ...
4    zagallo visita novo ct flamengo antes inaugura...
Name: doc, dtype: object

In [87]:
df.to_csv('noticias_processadas.csv', index=False)

In [84]:
vectorizer = TfidfVectorizer(max_features=20000)

tfidf_matrix = vectorizer.fit(df['doc'])
tfidf_matrix = vectorizer.transform(df['doc'])
tfidf_matrix

<1x147 sparse matrix of type '<class 'numpy.float64'>'
	with 147 stored elements in Compressed Sparse Row format>

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    tfidf_matrix, 
    df_target, 
    test_size=0.00, 
    random_state=1
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_train

(16314, 20000) (16314,)
(0, 20000) (0,)


<16314x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 2291713 stored elements in Compressed Sparse Row format>

In [8]:
# Modelo
m = XGBClassifier(
    n_jobs=4,
    random_state=1
)

In [80]:
# Validação local
np.mean(
    cross_val_score(
        m,
        X_train,
        y_train,
        cv=3,
        scoring='roc_auc'
    )
)

0.9995620410568785

In [16]:
# Gerando a previsão para o arquivo de teste
m.fit(
    X_train,
    y_train
)

In [117]:
noticia = '''Keyla falou por 10 minutos sobre o pedido de prisão do marido, negando sempre com veemência as acusações de abuso sexual apresentadas por centenas de mulheres contra João de Deus. Ela disse acreditar que tudo não passa de uma farsa, “um circo que montaram contra ele”.
– Eu acredito na inocência dele. Eu acho que é um grande circo que montaram contra ele, uma mentira. As pessoas que vêm aqui são pessoas com problemas. Elas tem problemas psicológicos. Todas resolveram dizer algo agora, mas sem fundamento. João é uma pessoa próxima, afetuosa. No momento certo a gente vai se pronunciar – afirmou ao GLOBO.
A mulher do líder espiritual estava visivelmente abalada. Falava no telefone a todo instante, enquanto uma babá segurava as crianças. Uma das ligações recebidas foi do próprio João de Deus. Ela disse a ele que as crianças, que a essa altura gritavam “papai”, queriam vê-lo.
Keyla disse que o médium tem a confiança dela e de todos que trabalham com ele na cidade. Ao avaliar a onda de denúncias de abuso sexual que se ergueu contra o marido, ela disse que "não sabia como tudo iria terminar", mas que ela "estaria do lado dele para tudo".'''

In [110]:
def pre_processar(noticia):
    global vectorizer
    
    txt = noticia
    
    return' '.join([t for t in word_tokenize(txt.lower()) if (t not in stopwords) and (t not in punctuation) and (not t.isdigit())])

In [58]:
df['doc'][-1:]

16313    razões quais bitcoin pode valer mil dólares br...
Name: doc, dtype: object

In [63]:
pred = m.predict_proba(pre_processar(noticia))[:,1]

                                                   0
0  keyla falou minutos sobre pedido prisão marido...


In [19]:
pd.DataFrame(pred)

,0


In [123]:
def calcular_noticia(noticia):
    df = pd.read_csv('noticias.csv')
    df = df.append(pd.DataFrame({"noticia": pre_processar(noticia)}, index=[0]), ignore_index=True)
    
    vectorizer = TfidfVectorizer(max_features=20000)

    tfidf_matrix = vectorizer.fit_transform(df['noticia'])
    X_train = tfidf_matrix[:-1]
    Y_train = df['target'][:-1]
    X_test = tfidf_matrix[-1:]
    
    # Modelo
    m = XGBClassifier(
        n_jobs=4,
        random_state=1
    )
    
    # Gerando a previsão para a linha de teste
    m.fit(
        X_train,
        Y_train
    )
    
    return m.predict_proba(X_test)[0,1]

In [124]:
calcular_noticia(noticia)

0.030224655